In [4]:
# Run an oxpy simulation from the main jupyter thread.
import oxpy
with oxpy.Context():
    manager = oxpy.OxpyManager('sim0/input')
    manager.run_complete()
    
# After running this, check nvidia-smi, there will be a python process remaining on the GPU
# It cannot be killed using `kill pid` from the command line
# Running `kill -9 pid` will also kill the Jupyter kernel.

In [10]:
# Function to pass to the pool
import oxpy
import multiprocessing as mp

def launch(inp):
    print("running:", inp, flush=True)
    with oxpy.Context():
        manager = oxpy.OxpyManager(inp)
        manager.run_complete()

In [11]:
# Run an oxpy simulation in a separate process
pool = mp.Pool(1)

try:
    response = [pool.apply_async(launch, (i,)) for i in ['sim0/input']]
    for i, r in enumerate(response):
        try:
            r.get(100000000)
        except KeyboardInterrupt:
            print("Caught KeyboardInterrupt1, terminating workers")
            pool.terminate()
            break
        except Exception as e:
            print("Error in simulation", i)
            print("Error was:", e)
            continue
        
except KeyboardInterrupt:
    print("Caught KeyboardInterrupt2, terminating workers")
    pool.terminate()
    pool.close()
    
pool.close()

# After running this normally, check nvidia-smi, you will not find the process
# If you're fast, you can interrupt the kernel while the simulation is running
# You have to do it twice to stop it (because of the subprocess?)
# If you stop it before the simulation finishes, there will be a zombie process left on the GPU.
# If you need more time, increase the `steps` parameter in `sim0/input`

running: sim0/input
